In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import random

import os
import pickle

In [9]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [10]:
# reading the files
infile = open('s01.dat', 'rb')
new_dict = pickle.load(infile, encoding="latin1")
infile.close()
labels=[]
data=[]

for keys in new_dict:
    if(keys=='labels'):
        labels=new_dict[keys]
    else:
        data=new_dict[keys]

#  LABELS 

In [11]:
#------------------------ labels ------------------------#



# The combinations of Valence and Arousal can be converted to emotional states:
    # High Arousal & Positive Valence (Excited, Happy)
    # Low Arousal & Positive Valence (Calm, Relaxed)
    # High Arousal & Negative Valence (Angry, Nervous)
    # Low Arousal & Negative Valence (Sad, Bored)
    
# Arousal can range from inactive (e.g. uninterested, bored) to active (e.g. alert, excited)
# valence ranges from unpleasant (e.g. sad, stressed) to pleasant (e.g. happy, elated)
# Dominance ranges from a helpless and weak feeling (without con- trol) to an empowered feeling (in control of everything).






# valence, arousal, dominance, liking
labels = np.array(labels)

# dealing with LABELS and creating a dataframe
valence = labels[:,0]
arousal = labels[:,1]
dominance = labels[:,2]
liking = labels[:,3]

results = pd.DataFrame({'valence': valence, 'arousal': arousal, 'dominance': dominance, 'liking': liking })
# results.describe()


valenceMedian = 4.5
arousalMedian = 4.5

# Function to check if each trial has positive or negative valence
def positive_valence(trial):
    return "positive" if results.iloc[trial]['valence'] >= valenceMedian else "negative"

# Function to check if each trial has high or low arousal
def high_arousal(trial):
    return "high" if results.iloc[trial]['arousal'] >= arousalMedian else "low"


# Convert ratings to high/low or positive/negative values
labels_encoded = []
for i in range (len(results)):
    labels_encoded.append([positive_valence(i), high_arousal(i)])
labels_encoded = np.reshape(labels_encoded, (40, 2))
labelsCategorized = pd.DataFrame(data=labels_encoded, columns=["valence", "arousal"])


# Dataset with only Valence column
df_valence = labelsCategorized['valence']

# Dataset with only Arousal column
df_arousal = labelsCategorized['arousal']


# High Arousal Positive Valence
highArousal_posValence = results[(results['valence'] >= results['valence'].median()) & (results['arousal'] >= results['arousal'].median())]

# Low Arousal Positive Valence dataset
lowArousal_posValence = results[(results['valence'] >= results['valence'].median()) & (results['arousal'] < results['arousal'].median())]

# High Arousal Negative Valence dataset
highArousal_negValence = results[(results['valence'] < results['valence'].median()) & (results['arousal'] >= results['arousal'].median())]

# Low Arousal Negative Valence dataset
lowArousal_negValence = results[(results['valence'] < results['valence'].median()) & (results['arousal'] < results['arousal'].median())]

# DATA

In [34]:
#------------------------ data ------------------------#
        

all_channels = np.array(['b1','b2', 'b3', 'b4', 'b5', 'b6','b7', 'b8', 'b9', 'b10','b11','b12', 'b13', 'b14', 'b15', 'b16','b17', 'b18', 'b19', 'b20','b21','b22', 'b23', 'b24', 'b25', 'b26','b27', 'b28', 'b29', 'b30', 'b31', 'b32', 'hEOG', 'vEOG', 'zEMG', 'tEMG', 'GSR', 'Respiration belt', 'Plethysmograph', 'Temp'])
peripheral_channels = np.array(["zEMG", "tEMG", "GSR", "Respiration belt", "Plethysmograph", "Temperature"])

# drop unneeded data 
data = data[:, 34:, :]

40
6
8064


In [ ]:
avgValence = 0;
avgArousal = 0;

for s in range(0, 100):
    print(s)
    # random videos will be taken out for testing (20%)
    randomIndexes = []

    count = 0;
    testPercentage = len(data)*20/100
    while( count < testPercentage ):
        randomNumber = random.randint(0,39) 
        if(not randomNumber in randomIndexes):
            randomIndexes.append(randomNumber)
            count = count+1
    
    
    #dividing train and test
    df_valence_result_test_temp = []
    df_valence_result_train_temp = []

    df_arousal_result_test_temp = []
    df_arousal_result_train_temp = []


    for i in range(0, len(df_valence)):
        if(i in randomIndexes):
            df_valence_result_test_temp.append(df_valence[i])
            df_arousal_result_test_temp.append(df_arousal[i])
        else:
            df_valence_result_train_temp.append(df_valence[i])
            df_arousal_result_train_temp.append(df_arousal[i])
            
    
    
    #repeating it 
    df_valence_result_test = []
    df_valence_result_train = []

    df_arousal_result_test = []
    df_arousal_result_train = []
    
    for i in range(0,len(df_valence_result_train_temp)):
        for j in range(0,8064):
            df_valence_result_train.append(df_valence_result_train_temp[i])
            df_arousal_result_train.append(df_arousal_result_train_temp[i])


    for i in range(0,len(df_valence_result_test_temp)):
        for j in range(0,8064):
            df_valence_result_test.append(df_valence_result_test_temp[i])
            df_arousal_result_test.append(df_arousal_result_test_temp[i])
            

            
            
    #DATA
    #convert it to dataframe
    df_data_train = pd.DataFrame(columns = peripheral_channels)
    df_data_test = pd.DataFrame(columns = peripheral_channels)
    df_dataPerVideo = pd.DataFrame()

    for i in range(0, len(data)): #video 40
        df_dataPerVideo = pd.DataFrame() 

        for j in range(0, len(data[i])): #channel 6
            temp = []
            for k in range(0, len(data[i][j])): #value 8064
                temp.append(data[i][j][k])
            df_dataPerVideo[peripheral_channels[j]] = temp    

        if(i in randomIndexes):
            df_data_test = df_data_test.append(df_dataPerVideo, ignore_index = True)
        else:
            df_data_train = df_data_train.append(df_dataPerVideo, ignore_index = True)

            
    
    # TRY FIRST VALENCE
    
    # apply standardization on numerical features
    sc = StandardScaler()
    standX_train = sc.fit_transform(df_data_train)
    standX_test = sc.transform(df_data_test)

    #Training and Predictions
#     classifier = KNeighborsClassifier(n_neighbors=5)
#     classifier = KNeighborsClassifier(n_neighbors=4)
#     classifier = LogisticRegression()
    classifier = RandomForestClassifier(n_estimators=100)
#     classifier = GaussianNB()
#     classifier = SVC(kernel='linear')
#     classifier = SVC(kernel='rbf')
    classifier.fit(standX_train, df_valence_result_train)

    #make predictions on our test data
    y_pred = classifier.predict(standX_test)

    #Evaluating the Algorithm
    avgValence = avgValence + accuracy_score(df_valence_result_test,y_pred)


    
    # TRY THEN AROUSAL
    
    # apply standardization on numerical features
    sc = StandardScaler()
    standX_train = sc.fit_transform(df_data_train)
    standX_test = sc.transform(df_data_test)

    #Training and Predictions
#     classifier = KNeighborsClassifier(n_neighbors=5)
#     classifier = KNeighborsClassifier(n_neighbors=4)
#     classifier = LogisticRegression()
    classifier = RandomForestClassifier(n_estimators=100)
#     classifier = GaussianNB()
#     classifier = SVC(kernel='linear')
#     classifier = SVC(kernel='rbf')
    classifier.fit(standX_train, df_arousal_result_train)

    #make predictions on our test data
    y_pred = classifier.predict(standX_test)

    #Evaluating the Algorithm
    avgArousal = avgArousal + accuracy_score(df_arousal_result_test,y_pred)
    
    
print("Valence accuracy for 100 times: ", avgValence)
print("Arousal accuracy for 100 times: ", avgArousal)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
